In [1]:
import json

from pathlib import Path

from kptncook.config import settings
from kptncook.models import Recipe
from kptncook.mealie import kptncook_to_mealie

from kptncook.mealie import MealieApiClient

In [2]:
example_path = Path.cwd().parent / "tests" / "fixtures" / "kptncook_example.json"
with example_path.open("r") as f:
    example = json.load(f)
recipe = Recipe.parse_obj(example)

In [3]:
recipe.ingredients[0].quantity

40.0

In [4]:
[i for i in recipe.ingredients if i.quantity is None]

[Ingredient(quantity=None, measure=None, ingredient=IngredientDetails(typ='basic', localized_title=LocalizedString(en='salt', de='Salz', es='sal', fr='sel', pt='sal'), uncountableTitle=LocalizedString(en='salt', de='Salz', es='sal', fr='sel', pt='sal'), category='SpicesSeasoning')),
 Ingredient(quantity=None, measure=None, ingredient=IngredientDetails(typ='basic', localized_title=LocalizedString(en='pepper', de='Pfeffer', es='pimienta', fr='poivre', pt='pimenta'), uncountableTitle=LocalizedString(en='pepper', de='Pfeffer', es='pimienta', fr='poivre', pt='pimenta'), category='SpicesSeasoning'))]

In [5]:
recipe.ingredients[0].measure

'g'

In [6]:
[ig.measure for ig in recipe.ingredients]

['g', None, 'g', 'EL', None, None, 'g', 'g', 'g', 'Zehe(n)', 'g', None, 'g']

In [7]:
mealie_recipe = kptncook_to_mealie(recipe)

In [8]:
mealie_recipe.image_url

'https://d2am1qai33sroc.cloudfront.net/image/636936644f000036005a5593?kptnkey=6q7QNKy-oIgk-IMuWisJ-jfN7s6'

In [9]:
# mealie_recipe.image_url

In [10]:
mealie_recipe.recipe_ingredient

[RecipeIngredient(title='Erbse', note='tiefgefroren', unit=Unit(name='g', id=None, description='', fraction=True, abbreviation=''), food=None, disable_amount=True, quantity=20.0),
 RecipeIngredient(title='Zitrone', note=None, unit=None, food=None, disable_amount=True, quantity=0.25),
 RecipeIngredient(title='Dill', note='frisch', unit=Unit(name='g', id=None, description='', fraction=True, abbreviation=''), food=None, disable_amount=True, quantity=2.5),
 RecipeIngredient(title='Olivenöl', note=None, unit=Unit(name='EL', id=None, description='', fraction=True, abbreviation=''), food=None, disable_amount=True, quantity=0.5),
 RecipeIngredient(title='Salz', note=None, unit=None, food=None, disable_amount=True, quantity=None),
 RecipeIngredient(title='Pfeffer', note=None, unit=None, food=None, disable_amount=True, quantity=None),
 RecipeIngredient(title='Muschelnudel', note='groß', unit=Unit(name='g', id=None, description='', fraction=True, abbreviation=''), food=None, disable_amount=True, 

# Import via Api

In [11]:
client = MealieApiClient("http://localhost:9000/api")
client.login(username="jochen")

 ········


In [12]:
%%time
client.create_recipe(mealie_recipe)

uberbackene-muschelnudeln-mit-lachs-senf-dill-sauce
{"id":"f53c8bfc-0f66-44ad-97f6-a9f6f327a5c0","userId":"d1b5874c-4815-43a6-bba9-5c9b3b961810","groupId":"5aa776c9-b1af-4713-9e92-e563153a96a0","name":"Überbackene Muschelnudeln mit Lachs & Senf-Dill-Sauce","slug":"uberbackene-muschelnudeln-mit-lachs-senf-dill-sauce","image":null,"recipeYield":"1 Portionen","totalTime":null,"prepTime":"25","cookTime":"10","performTime":null,"description":"","recipeCategory":[],"tags":[{"id":"711c5c31-6a18-4f9e-8c4e-488808dd1518","name":"kptncook","slug":"kptncook"}],"tools":[],"rating":null,"orgURL":null,"recipeIngredient":[{"title":"Erbse","note":"tiefgefroren","unit":{"name":"g","description":"","extras":{},"fraction":true,"abbreviation":"","useAbbreviation":false,"id":"ce5d051b-b560-4652-8f75-9d803bed9683","createdAt":"2022-12-03T10:11:39.902417","updateAt":"2022-12-03T10:11:39.902422"},"food":null,"disableAmount":true,"quantity":20.0,"originalText":null,"referenceId":"07fdef1c-a36f-4627-8de5-a462feb

Recipe(id=UUID('f53c8bfc-0f66-44ad-97f6-a9f6f327a5c0'), user_id=UUID('d1b5874c-4815-43a6-bba9-5c9b3b961810'), group_id=UUID('5aa776c9-b1af-4713-9e92-e563153a96a0'), name='Überbackene Muschelnudeln mit Lachs & Senf-Dill-Sauce', slug='uberbackene-muschelnudeln-mit-lachs-senf-dill-sauce', image=None, recipe_yield=None, total_time=None, prep_time=None, cook_time=None, perform_time=None, description='', recipe_category=[], tags=[RecipeTag(name='kptncook', slug='kptncook', group_id=None, id=UUID('711c5c31-6a18-4f9e-8c4e-488808dd1518'))], tools=[], rating=None, org_url=None, recipe_ingredient=[], date_added=None, date_updated=None, recipe_instructions=[], nutrition=Nutrition(calories='900', fat_content=None, protein_content=None, carbohydrate_content=None, fiber_content=None, sodium_content=None, sugar_content=None), settings=RecipeSettings(public=True, show_nutrition=True, show_assets=False, landscape_view=False, disable_comments=False, disable_amount=False, locked=False), assets=[], notes=[

In [13]:
[ig for ig in mealie_recipe.recipe_ingredient if ig.note is not None]

[RecipeIngredient(title='Erbse', note=' tiefgefroren', unit=Unit(id=None, name='g', description='', fraction=True, abbreviation=''), food=None, disable_amount=True, quantity=20.0),
 RecipeIngredient(title='Dill', note=' frisch', unit=Unit(id=None, name='g', description='', fraction=True, abbreviation=''), food=None, disable_amount=True, quantity=2.5),
 RecipeIngredient(title='Muschelnudel', note=' groß', unit=Unit(id=None, name='g', description='', fraction=True, abbreviation=''), food=None, disable_amount=True, quantity=40.0),
 RecipeIngredient(title='Lachsfilet', note=' tiefgefroren', unit=Unit(id=None, name='g', description='', fraction=True, abbreviation=''), food=None, disable_amount=True, quantity=50.0)]

In [11]:
mealie_recipe.recipe_instructions

[RecipeStep(title=None, text='Alles parat?', ingredientReferences=[], image=Image(name='REZ_3062_01.jpg', type='step', url='https://d2am1qai33sroc.cloudfront.net/image/63653b8d4b00007500b0c51d')),
 RecipeStep(title=None, text='Lachsfilets bei Bedarf auftauen.', ingredientReferences=[], image=Image(name='REZ_3062_18.jpg', type=None, url='https://d2am1qai33sroc.cloudfront.net/image/635a6d265100003600061cf6')),
 RecipeStep(title=None, text='Pasta nach Packungsanleitung zubereiten.', ingredientReferences=[], image=Image(name='REZ_3062_03.jpeg', type='step', url='https://d2am1qai33sroc.cloudfront.net/image/63653b8d4b00007500b0c51e')),
 RecipeStep(title=None, text='Währenddessen etwas Olivenöl in einer Pfanne erhitzen und Lachsfilet von beiden Seiten bei mittlerer Hitze ca. 2-3 min. braten. Mit Salz und Pfeffer würzen und zur Seite stellen.', ingredientReferences=[], image=Image(name='REZ_3062_04.jpg', type='step', url='https://d2am1qai33sroc.cloudfront.net/image/63653b8d4b00007500b0c51f')),

In [10]:
mealie_recipe.tags

[]

# Get all KptnCook Recipes

In [9]:
recipes = client.get_all_recipes()

In [10]:
recipes_with_details = []
for recipe in recipes:
    recipes_with_details.append(client.get_via_slug(recipe.slug))

In [11]:
recipes_with_details[0].extras

{'kptncook_id': '5e5390e2740000cdf1381c64', 'source': 'kptncook'}

In [12]:
kc_recipes = [r for r in recipes_with_details if r.extras.get("source") == "kptncook"]

In [13]:
kc_recipes

[Recipe(id=UUID('756c3a9c-739d-4598-b96d-7a1f878177a4'), user_id=UUID('899979e8-f240-4d53-8ed3-eaeac21ecd53'), group_id=UUID('71aae446-1aa5-4db7-9782-4021692cd0ce'), name='Chili sin Carne', slug='chili-sin-carne', image=None, recipe_yield=None, total_time=None, prep_time=None, cook_time=None, perform_time=None, description='', recipe_category=[], tags=[RecipeTag(name='kptncook')], tools=[], rating=None, org_url=None, recipe_ingredient=[], date_added=None, date_updated=None, recipe_instructions=[], nutrition=Nutrition(calories='835', fat_content=None, protein_content=None, carbohydrate_content=None, fiber_content=None, sodium_content=None, sugar_content=None), settings=RecipeSettings(public=False, show_nutrition=False, show_assets=False, landscape_view=False, disable_comments=True, disable_amount=True, locked=False), assets=[], notes=[RecipeNote(title='author comment', text='Hot stuff')], extras={'kptncook_id': '5e5390e2740000cdf1381c64', 'source': 'kptncook'})]

# Sync

In [14]:
from kptncook.repositories import RecipeRepository

In [15]:
repo = RecipeRepository(settings.root)

In [17]:
api_recipes = []
for repo_recipe in repo.list():
    recipe = Recipe.parse_obj(repo_recipe.data)
    mealie_recipe = kptncook_to_mealie(recipe)
    api_recipes.append(mealie_recipe)

In [18]:
len(api_recipes)

6

In [19]:
api_recipes[0].extras

{'kptncook_id': '5e5390e2740000cdf1381c64', 'source': 'kptncook'}

In [22]:
ids_in_mealie = {r.extras["kptncook_id"] for r in kc_recipes}
ids_from_api = {r.extras["kptncook_id"] for r in api_recipes}

In [25]:
ids_to_add = ids_from_api - ids_in_mealie

In [26]:
recipes_to_add = []
for recipe in api_recipes:
    if recipe.extras.get("kptncook_id") in ids_to_add:
        recipes_to_add.append(recipe)

In [27]:
len(recipes_to_add)

5

In [28]:
for recipe in recipes_to_add:
    created = client.create_recipe(recipe)